In [ ]:
"""
This will create a global grid of the approximate size of each grid square.
"""
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import math
import rasterio as rs

In [ ]:
def create_xarray(lats,lons,data):

	ds = xr.Dataset(
			data_vars=dict(
					population=(["lat", "lon"], data)),  
			coords=dict(
					lon=("lon", lons),
					lat=("lat", lats),
			))
	ds.lat.attrs['units'] = "degrees_north"
	ds.lon.attrs['units'] = "degrees_east"
	ds.population.assign_attrs(units="count")
	return ds

In [ ]:
tif1 = '/user/home/al18709/work/population/ppp_2020_1km_Aggregated.tif'
tif2 = '/user/home/al18709/work/population/SSP2_2085.tif'

dataset1 = rs.open(tif1)
pop1 = dataset1.read(1)

dataset2 = rs.open(tif2)
pop2 = dataset2.read(1)

In [ ]:
print(pop1.shape)

In [ ]:
plt.imshow(pop1)

In [ ]:
print(pop1)

In [ ]:
msk = dataset2.read_masks(1)
print(dataset2.nodata)
msk1 = dataset1.read_masks(1)
# print(dataset2.read())
print(np.sum(msk == 1))
print(np.sum(msk1 == 1))

In [ ]:
print(pop1.shape)
print(pop2.shape)

In [ ]:
population_1km_file = '/user/home/al18709/work/population/SSP2_2085.nc'
population_10km_file = '/user/home/al18709/work/population/SSP2_2085_10km.nc'

population_1km = xr.open_dataset(population_1km_file)
population_10km = xr.open_dataset(population_10km_file)

In [ ]:
lats = population_1km.Band1.lat.values
lons = population_1km.Band1.lon.values

In [ ]:
pop_2020 = create_xarray(lats,lons,pop1)
pop_2085 = create_xarray(lats,lons,pop2)

In [ ]:
pop_2020.to_netcdf('/user/home/al18709/work/population/ppp_2020_1km_Aggregated_final.nc',format='netCDF4')

pop_2085.to_netcdf('/user/home/al18709/work/population/SSP2_2085_final.nc',format='netCDF4')

In [ ]:
population_1km_file = '/user/home/al18709/work/population/ppp_2020_1km_Aggregated_final.nc'
population_10km_file = '/user/home/al18709/work/population/ppp_2020_10km_Aggregated_final.nc'

population_1km = xr.open_dataset(population_1km_file)
population_10km = xr.open_dataset(population_10km_file)

In [ ]:
population_1km.population.values[population_1km.population.values <= -3.4028235e+37] = 0
population_10km.population.values[population_10km.population.values <= -3.4028235e+37] = 0

In [ ]:
lats_1km = population_1km.lat
lons_1km = population_1km.lon

lats_10km = population_10km.lat
lons_10km = population_10km.lon

In [ ]:
area_1km = np.abs(lats_1km[0] - lats_1km[1]) * np.abs(lons_1km[0] - lons_1km[1])

area_10km = np.abs(lats_10km[0] - lats_10km[1]) * np.abs(lons_10km[0] - lons_10km[1])

In [ ]:
print(area_1km)
print(area_10km)
print(area_10km / area_1km)

In [ ]:
print(np.sum(population_1km.population.values !=0))
print(np.sum(population_10km.population.values !=0))

In [ ]:
pop_total_1km = np.sum(population_1km.population.values)
pop_total_10km = np.sum(population_10km.population.values)
pop_total_10km_corrected = pop_total_10km * 144

print(pop_total_1km / pop_total_10km)
print(pop_total_10km * 144)
print(pop_total_1km)

print(pop_total_10km * 144 / pop_total_1km * 100 - 100)